In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from ucimlrepo import fetch_ucirepo 

# Fetch dataset
heart_disease = fetch_ucirepo(id=45)
print("Raw features data:", heart_disease.data.features[:5]) 

Raw features data:    age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   1       145   233    1        2      150      0      2.3      3   
1   67    1   4       160   286    0        2      108      1      1.5      2   
2   67    1   4       120   229    0        2      129      1      2.6      2   
3   37    1   3       130   250    0        0      187      0      3.5      3   
4   41    0   2       130   204    0        2      172      0      1.4      1   

    ca  thal  
0  0.0   6.0  
1  3.0   3.0  
2  2.0   7.0  
3  0.0   3.0  
4  0.0   3.0  


In [12]:
#2(5 pts) Clean your dataset to turn categorical values into numerical ones. One-hot encoding is likely the answer, but it depends on the dataset. Your data may have ordinal columns, for example where one-hot encoding is not as appropriate.
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from ucimlrepo import fetch_ucirepo 

heart_disease = fetch_ucirepo(id=45)
X = pd.DataFrame(heart_disease.data.features, columns=heart_disease.data.feature_names)
y = pd.Series(heart_disease.data.targets.squeeze(), name='Outcome')
class_labels = LabelEncoder()
X['sex'] = class_labels.fit_transform(X['sex'].values)
X['cp'] = class_labels.fit_transform(X['cp'].values)
X['chol'] = class_labels.fit_transform(X['chol'].values)
X['fbs'] = class_labels.fit_transform(X['fbs'].values)
X['restecg'] = class_labels.fit_transform(X['restecg'].values)
X['thalach'] = class_labels.fit_transform(X['thalach'].values)
X['exang'] = class_labels.fit_transform(X['exang'].values)
X['oldpeak'] = class_labels.fit_transform(X['oldpeak'].values)
X['slope'] = class_labels.fit_transform(X['slope'].values)
X['ca'] = class_labels.fit_transform(X['ca'].values)
X['thal'] = class_labels.fit_transform(X['thal'].values)

print(X.head())


   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   0       145    64    1        2       49      0       22      2   
1   67    1   3       160   111    0        2       10      1       15      1   
2   67    1   3       120    60    0        2       29      1       25      1   
3   37    1   2       130    80    0        0       84      0       32      2   
4   41    0   1       130    35    0        2       71      0       14      0   

   ca  thal  
0   0     1  
1   3     0  
2   2     2  
3   0     0  
4   0     0  


In [28]:
#3(3 pts) Perform univariate linear regression on the dataset. Select your variable to predict. How well did this model perform? Is this a good approach for this dataset? Why or why not? 
import numpy as np
import pandas as pd
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
heart_disease = fetch_ucirepo(id=45)
X = pd.DataFrame(heart_disease.data.features, columns=heart_disease.data.feature_names)
y = pd.Series(heart_disease.data.targets.to_numpy().flatten(), name='Outcome')  
valid_indices = ~y.isna()  
x_valid = X.loc[valid_indices, 'age'].astype(float).values.ravel()  # 1D array for features
y_valid = y[valid_indices].values.ravel()  # 1D array for target
scaler = StandardScaler()
x_valid = scaler.fit_transform(x_valid.reshape(-1, 1)).ravel()  # Reshape for scaling, then flatten

class LinearRegressor:
    def __init__(self, x, y, alpha=0.001, b0=0, b1=0):  # Reduce alpha
        self.i = 0
        self.x = x
        self.y = y
        self.alpha = alpha
        self.b0 = b0
        self.b1 = b1
        if len(x) != len(y):
            raise TypeError("x and y should have the same number of rows.")

    def cost_derivative(self):
        errors = [(self.predict(xi) - yi) ** 2 for xi, yi in zip(self.x, self.y)]
        mse = sum(errors) / len(errors)
        return mse

    def predict(self, x):
        return self.b0 + self.b1 * x

    def fit(self):
        max_epochs = 1000
        for _ in range(max_epochs):
            self.update_coeff(0)
            self.update_coeff(1)
        return self.cost_derivative()  # Return the MSE after fitting

    def update_coeff(self, i):
        if i == 0:
            self.b0 -= self.alpha * self.grad_fun(0)
        elif i == 1:
            self.b1 -= self.alpha * self.grad_fun(1)

    def grad_fun(self, i):
        return sum([
            2 * (self.predict(xi) - yi) * (1 if i == 0 else xi)
            for xi, yi in zip(self.x, self.y)
        ]) / len(self.x)
regressor = LinearRegressor(x=x_valid, y=y_valid)  
mse = regressor.fit()  
print("Mean Squared Error:", mse)
#all code is adapted from geeks for geeks

Mean Squared Error: 1.4469980118591779


In [30]:
#3(8 pts) Perform KNN on this dataset. As part of this, write a function that selects the optimal value of k. How well did this model perform?
#The model performed not too well not to bad because it was 0.60 a little better than 50 but not significant.
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import pandas as pd
from ucimlrepo import fetch_ucirepo
heart_disease = fetch_ucirepo(id=45)
X = pd.DataFrame(heart_disease.data.features, columns=heart_disease.data.feature_names)
y = heart_disease.data.targets.squeeze()
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_standardized = scaler.fit_transform(X_train)
X_test_standardized = scaler.transform(X_test)
def select_optimal_k(X_train, y_train, X_test, y_test, max_k):
    accuracy_list = []
    
    for k in range(1, max_k + 1):
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracy_list.append(accuracy)
        print(f'Accuracy for k={k}: {accuracy:.4f}')

    optimal_k = accuracy_list.index(max(accuracy_list)) + 1
    print(f'Optimal k: {optimal_k} with accuracy: {max(accuracy_list):.4f}')
    return optimal_k
optimal_k = select_optimal_k(X_train_standardized, y_train, X_test_standardized, y_test, max_k=20)
knn = KNeighborsClassifier(n_neighbors=optimal_k)
knn.fit(X_train_standardized, y_train)
y_pred = knn.predict(X_test_standardized)
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy with KNN (optimal k):", accuracy)

#adapted from geeks for geeks


Accuracy for k=1: 0.5082
Accuracy for k=2: 0.5246
Accuracy for k=3: 0.4918
Accuracy for k=4: 0.5246
Accuracy for k=5: 0.5246
Accuracy for k=6: 0.5574
Accuracy for k=7: 0.5410
Accuracy for k=8: 0.5410
Accuracy for k=9: 0.5574
Accuracy for k=10: 0.5738
Accuracy for k=11: 0.6066
Accuracy for k=12: 0.5738
Accuracy for k=13: 0.5574
Accuracy for k=14: 0.5574
Accuracy for k=15: 0.5574
Accuracy for k=16: 0.5410
Accuracy for k=17: 0.5574
Accuracy for k=18: 0.5574
Accuracy for k=19: 0.5410
Accuracy for k=20: 0.5738
Optimal k: 11 with accuracy: 0.6066
Accuracy with KNN (optimal k): 0.6065573770491803


In [29]:
#4(6 pts) Work with your dataset to perform logistic regression. How well did this perform?
#It has a 60% value so I don't think it means very good nor very bad, so not statistically significant.
from ucimlrepo import fetch_ucirepo
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

def load_and_prepare_data():
    """Load the UC Irvine Heart Disease dataset and preprocess it."""
    heart_disease = fetch_ucirepo(id=45)
    X = pd.DataFrame(heart_disease.data.features, columns=heart_disease.data.feature_names)
    y = heart_disease.data.targets.squeeze()  # Convert target to 1D
    return X, y

def preprocess_data(X):
    """Handle missing values and scale features."""
    imputer = SimpleImputer(strategy='mean')  
    X_imputed = imputer.fit_transform(X)
    sc = StandardScaler()
    X_scaled = sc.fit_transform(X_imputed)
    
    return X_scaled

def train_logistic_regression(X_train, y_train):
    """Train the Logistic Regression model."""
    model = LogisticRegression(random_state=42)
    model.fit(X_train, y_train)
    return model

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    confusion = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    print("Accuracy:", accuracy)
    

def main():
    X, y = load_and_prepare_data()
    X_scaled = preprocess_data(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

    model = train_logistic_regression(X_train, y_train)
    evaluate_model(model, X_test, y_test)

if __name__ == "__main__":
    main()
    


Accuracy: 0.6065573770491803


In [16]:
#5. (3 pts) Perform normalization on your dataset. Does it change the performance for 2-4? What is the best measure of performance for your dataset (accuracy or something else) and why?
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import pandas as pd
from ucimlrepo import fetch_ucirepo
heart_disease = fetch_ucirepo(id=45)
X = pd.DataFrame(heart_disease.data.features, columns=heart_disease.data.feature_names)
y = heart_disease.data.targets.squeeze()
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42, stratify=y)
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)
def train_and_evaluate_model(X_train, X_test, y_train, y_test):
    model = LogisticRegression(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    confusion = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred, zero_division=1)
    print("Accuracy with normalization:", accuracy)
    print("Confusion Matrix (normalized):\n", confusion)
    print("Classification Report (normalized):\n", report)
train_and_evaluate_model(X_train_normalized, X_test_normalized, y_train, y_test)


#Yes the performance does change with different classes. The f1-score changes with different classifications because the data that comes from classification can vary, 
#each class doesn't have the same data size.Also there can be false negatives's and positives. It's one of the biggest issues with classification reports,
#but that is why we used normalization to eliminate that. The F1 score is the gold standard for performance/accuracy. All code was adapted from Joel grus network analysis and I used codes for geeks for the squeeze. 


Accuracy with normalization: 0.6065573770491803
Confusion Matrix (normalized):
 [[31  1  1  0  0]
 [ 6  4  1  0  0]
 [ 2  1  0  4  0]
 [ 0  1  4  2  0]
 [ 0  2  0  1  0]]
Classification Report (normalized):
               precision    recall  f1-score   support

           0       0.79      0.94      0.86        33
           1       0.44      0.36      0.40        11
           2       0.00      0.00      0.00         7
           3       0.29      0.29      0.29         7
           4       1.00      0.00      0.00         3

    accuracy                           0.61        61
   macro avg       0.51      0.32      0.31        61
weighted avg       0.59      0.61      0.57        61

